In [2]:
import pandas as pd
import os
import requests as r

In [3]:
path = "/home/robbe/ionbot/ionbot_0.9.5/PXD016278/ionbot_files"
subdirs = [dirname for dirname in os.listdir(path)]

In [4]:
Dioxidated_proteins = set()
DioxidatedW_proteins = set()
Dioxidated_proteins_controls = set()
DioxidatedW_proteins_controls = set()
for subdir in subdirs:
    if "_AD2_" in subdir:
        with open(path + "/" + subdir + "/ionbot.modifications.csv") as fn:
            for line in fn.readlines():
                linelist = line.split(",")
                unexpected_modification = linelist[2]
                protein = linelist[1]
                if "Dioxidation" in unexpected_modification and not "sp" in protein:
                    Dioxidated_proteins.add(protein)
                if "Dioxidation[W]" in unexpected_modification and not "sp" in protein:
                    DioxidatedW_proteins.add(protein)
    if "_ctrl2_" in subdir:
        with open(path + "/" + subdir + "/ionbot.modifications.csv") as fn:
            for line in fn.readlines():
                linelist = line.split(",")
                unexpected_modification = linelist[2]
                protein = linelist[1]
                if "Dioxidation" in unexpected_modification and not "sp" in protein:
                    Dioxidated_proteins_controls.add(protein)
                if "Dioxidation[W]" in unexpected_modification and not "sp" in protein:
                    DioxidatedW_proteins_controls.add(protein)

Dioxidated_proteins_controlled = Dioxidated_proteins - Dioxidated_proteins_controls
DioxidatedW_proteins_controlled = DioxidatedW_proteins - DioxidatedW_proteins_controls

See file for functional_enrichment for dioxidation[W] proteins

In [5]:
Dioxidated_proteins = {}
DioxidatedW_proteins = {}
Dioxidated_proteins_controls = {}
DioxidatedW_proteins_controls = {}
for subdir in subdirs:
    if "_AD2_" in subdir:
        with open(path + "/" + subdir + "/ionbot.modifications.csv") as fn:
            for line in fn.readlines():
                linelist = line.split(",")
                unexpected_modification = linelist[2]
                protein = linelist[1]
                position = linelist[3]
                if "Dioxidation" in unexpected_modification and not "sp" in protein:
                    Dioxidated_proteins[protein] = position
                if "Dioxidation[W]" in unexpected_modification and not "sp" in protein:
                    DioxidatedW_proteins[protein] = position
    if "_ctrl2_" in subdir:
        with open(path + "/" + subdir + "/ionbot.modifications.csv") as fn:
            for line in fn.readlines():
                linelist = line.split(",")
                unexpected_modification = linelist[2]
                protein = linelist[1]
                position = linelist[3]
                if "Dioxidation" in unexpected_modification and not "sp" in protein:
                    Dioxidated_proteins_controls[protein] = position
                if "Dioxidation[W]" in unexpected_modification and not "sp" in protein:
                    DioxidatedW_proteins_controls[protein] = position

Dioxidated_proteins_set = {}
DioxidatedW_proteins_set = {}
Dioxidated_proteins_controls_set = {}
DioxidatedW_proteins_controls_set = {}
Dioxidated_proteins_controlled_set = {}
DioxidatedW_proteins_controlled_set = {}

for key,value in Dioxidated_proteins.items():
    if value not in Dioxidated_proteins_set.values():
        Dioxidated_proteins_set[key] = value

for key,value in DioxidatedW_proteins.items():
    if value not in DioxidatedW_proteins_set.values():
        DioxidatedW_proteins_set[key] = value       

for key,value in Dioxidated_proteins_controls.items():
    if value not in Dioxidated_proteins_controls_set.values():
        Dioxidated_proteins_controls_set[key] = value    

for key,value in DioxidatedW_proteins_controls.items():
    if value not in DioxidatedW_proteins_controls_set.values():
        DioxidatedW_proteins_controls_set[key] = value  

for key,value in Dioxidated_proteins.items():
    if value not in Dioxidated_proteins_controls_set.values():
        Dioxidated_proteins_controlled_set[key] = value

for key,value in DioxidatedW_proteins.items():
    if value not in DioxidatedW_proteins_controls_set.values():
        DioxidatedW_proteins_controlled_set[key] = value

In [6]:
with open("Dioxidated_proteins.txt", "w") as fn:
    fn.write("Protein\tCenterPosition\n")
    for key, value in Dioxidated_proteins_controlled_set.items():
        fn.write("{}\t{}\n".format(key, value))

    fn.close()

In [7]:
with open("DioxidatedW_proteins.txt", "w") as fn:
    fn.write("Protein\tCenterPosition\n")
    for key, value in DioxidatedW_proteins_controlled_set.items():
        fn.write("{}\t{}\n".format(key, value))

    fn.close()

In [8]:
Kynureninprots = {}
for subdir in subdirs:
    if "_AD2_" in subdir:
        with open(path + "/" + subdir + "/ionbot.modifications.csv") as fn:
            for line in fn.readlines():
                linelist = line.split(",")
                unexpected_modification = linelist[2]
                protein = linelist[1]
                position = linelist[3]
                if "Trp->Kynurenin[W]" in unexpected_modification and not "sp" in protein:
                    Kynureninprots[protein] = position

Kynureninprots_set = {}
for key,value in Kynureninprots.items():
    if value not in Kynureninprots_set.values():
        Kynureninprots_set[key] = value

with open("Kynureninprots_proteins.txt", "w") as fn:
    fn.write("Protein\tCenterPosition\n")
    for key, value in Kynureninprots_set.items():
        fn.write("{}\t{}\n".format(key, value))

    fn.close()